### Calculate trades based on moving averages cross

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]

In [3]:
df = pd.read_pickle(utils.get_data_filename(pair, granularity, True))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [6]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [7]:
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [8]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-12-21T19:00:00.000000000Z,1.09898,1.10010,1.09884,1.09992,1.097076,1.096099
1,2023-12-21T20:00:00.000000000Z,1.09992,1.10032,1.09929,1.10031,1.097413,1.096224
2,2023-12-21T21:00:00.000000000Z,1.10030,1.10127,1.10027,1.10122,1.097773,1.096355
3,2023-12-21T22:00:00.000000000Z,1.10102,1.10106,1.09980,1.10088,1.098172,1.096482
4,2023-12-21T23:00:00.000000000Z,1.10088,1.10126,1.10057,1.10057,1.098538,1.096603


In [9]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)


In [10]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV
0,2023-12-21T19:00:00.000000000Z,1.09898,1.10010,1.09884,1.09992,1.097076,1.096099,0.000977,NaN
1,2023-12-21T20:00:00.000000000Z,1.09992,1.10032,1.09929,1.10031,1.097413,1.096224,0.001189,0.000977
2,2023-12-21T21:00:00.000000000Z,1.10030,1.10127,1.10027,1.10122,1.097773,1.096355,0.001418,0.001189
3,2023-12-21T22:00:00.000000000Z,1.10102,1.10106,1.09980,1.10088,1.098172,1.096482,0.001690,0.001418
4,2023-12-21T23:00:00.000000000Z,1.10088,1.10126,1.10057,1.10057,1.098538,1.096603,0.001935,0.001690


In [11]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [12]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [16]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2023-12-21T19:00:00.000000000Z,1.09898,1.10010,1.09884,1.09992,1.097076,1.096099,0.000977,NaN,False
1,2023-12-21T20:00:00.000000000Z,1.09992,1.10032,1.09929,1.10031,1.097413,1.096224,0.001189,0.000977,False
2,2023-12-21T21:00:00.000000000Z,1.10030,1.10127,1.10027,1.10122,1.097773,1.096355,0.001418,0.001189,False
3,2023-12-21T22:00:00.000000000Z,1.10102,1.10106,1.09980,1.10088,1.098172,1.096482,0.001690,0.001418,False
4,2023-12-21T23:00:00.000000000Z,1.10088,1.10126,1.10057,1.10057,1.098538,1.096603,0.001935,0.001690,False


In [13]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
105,2023-12-29T04:00:00.000000000Z,1.10654,1.10688,1.10638,1.10668,1.107342,1.107549,-0.000207,0.000163,True
193,2024-01-04T20:00:00.000000000Z,1.09470,1.09494,1.09431,1.09494,1.094825,1.094747,0.000078,-0.000179,True
207,2024-01-05T10:00:00.000000000Z,1.09131,1.09138,1.09037,1.09129,1.093649,1.093685,-0.000036,0.000124,True
219,2024-01-07T22:00:00.000000000Z,1.09487,1.09487,1.09431,1.09454,1.093556,1.093547,0.000010,-0.000121,True
263,2024-01-09T18:00:00.000000000Z,1.09339,1.09402,1.09224,1.09242,1.094252,1.094348,-0.000096,0.000110,True


In [18]:
df_trades.shape

(75, 10)

In [19]:
df_plot = df_ma.iloc[50:250].copy()

In [20]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
    ))
fig.update_layout(width=1200,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()